<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Fine_tune_LLAMA_vi_instruction_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune Llama 3.2 on Vietnamese Instruct dataset

![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/7f10d7c0-bde7-11ef-a3aa-4b5f419a5062-LLMs.png)

### Training

In [1]:
!pip install unsloth

!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-n0qljua_/unsloth_b67c7584f42d406f9e878fba03ee40d2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-n0qljua_/unsloth_b67c7584f42d406f9e878fba03ee40d2
  Resolved https://github.com/unslothai/unsloth.git to commit 5dddf27f3ba94506c48251e907031039eecd40d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.5-py3-none-any.whl size=176838 sha256=98f80579f059ea5bd9e3c541ea46bc2fd3472465516eb6a4d5acade4e1407209
  Stored in directory: /tmp/pip-ephem-wheel-cache-o9dn5azp/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import random

from datasets import load_dataset
open_instruct_dataset = load_dataset("5CD-AI/Vietnamese-Ecommerce-Alpaca", split="train")

ecommerce_alpaca.json:   0%|          | 0.00/84.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69303 [00:00<?, ? examples/s]

In [5]:
open_instruct_dataset[1]

{'output': 'Sử dụng túi giặt có thể mang lại một số lợi ích, bao gồm:\n\n- Bảo vệ quần áo: Túi giặt có thể giúp bảo vệ quần áo khỏi bị hư hỏng trong quá trình giặt, đặc biệt là đối với quần áo mỏng manh hoặc dễ bị rách.\n\n- Giúp giặt sạch hơn: Túi giặt có thể giúp quần áo được giặt sạch hơn bằng cách tạo ra chuyển động tốt hơn trong máy giặt và giúp chất tẩy rửa tiếp xúc với quần áo tốt hơn.\n\n- Giảm xoắn rối: Túi giặt có thể giúp giảm xoắn rối quần áo trong quá trình giặt, giúp quần áo dễ ủi hơn và trông đẹp hơn.\n\n- Tiết kiệm thời gian: Túi giặt có thể giúp tiết kiệm thời gian bằng cách giảm thời gian ủi quần áo và giúp quần áo khô nhanh hơn.\n\n- Giảm tiếng ồn: Túi giặt có thể giúp giảm tiếng ồn từ máy giặt, đặc biệt là đối với máy giặt cũ hoặc bị hỏng.',
 'instruction': 'Nói cho tôi biết tại sao nên sử dụng túi giặt.',
 'input': ''}

In [6]:
max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [7]:
EOS_TOKEN = tokenizer.eos_token

In [8]:
def format_row_as_instruction_prompt(example):
    has_input = example.get('input', None) is not None

    if has_input:
        primer_prompt = ("Below is an instruction that describes a task, paired with an input "
                         "that provides further context. Write a response that appropriately completes the request.")
        input_template = f"### Input: \n{example['input']}\n\n"
    else:
        primer_prompt = ("Below is an instruction that describes a task. "
                         "Write a response that appropriately completes the request.")
        input_template = ""

    instruction_template = f"### Instruction: \n{example['instruction']}\n\n"

    if example.get('output', None):
        response_template = f"### Response: \n{example['output']}\n\n"
    else:
        response_template = ""

    # Wrap the resulting string in a list
    return f"{primer_prompt}\n\n{instruction_template}{input_template}{response_template}"


def formatting_prompt(examples):
    """
    Formats a batch of examples using the format_row_as_instruction_prompt logic.

    Args:
        examples (dict): A dictionary with keys "instruction", "input", and "output".

    Returns:
        dict: A dictionary containing a list of formatted text prompts.
    """
    formatted_texts = []

    for instruction, input_, output in zip(
        examples.get("instruction", []),
        examples.get("input", []),
        examples.get("output", []),
    ):
        # Create a single example dictionary
        example = {
            "instruction": instruction,
            "input": input_,
            "output": output
        }

        # Use format_row_as_instruction_prompt to format the example
        formatted_text = format_row_as_instruction_prompt(example) + EOS_TOKEN
        formatted_texts.append(formatted_text)

    return {"text": formatted_texts}

In [9]:
training_data = open_instruct_dataset.map(formatting_prompt, batched=True)

Map:   0%|          | 0/69303 [00:00<?, ? examples/s]

In [10]:
training_data[0]["text"]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: \nSử dụng cánh tay robot để nhặt một nồi điện cụ thể và di chuyển nó đến một vị trí cụ thể. Nồi điện đó là nồi cơm điện màu đỏ, vị trí là tủ bếp gần đó.\n\n### Input: \n\n\n### Response: \nĐược thôi! Đầu tiên, cánh tay robot sẽ thu thập thông tin về tọa độ của vị trí hiện tại của nồi cơm điện màu đỏ và tủ bếp gần đó. Khi thông tin đã được nhập, cánh tay robot sẽ xoay về phía nồi cơm điện, kéo dài cánh tay để chạm đến nồi và đóng chốt xung quanh để giữ nó chắc chắn. Sau đó, cánh tay robot sẽ được kéo lại trong khi vẫn giữ chắc nồi cơm điện và cuối cùng xoay về phía tủ bếp gần đó. Cánh tay robot sẽ được hạ xuống để đặt nồi cơm điện vào tủ bếp và chốt được thả ra để bỏ nồi cơm điện đi. Sau đó, cánh tay robot sẽ thu lại vị trí ban đầu.\n\n<|end_of_text|>'

### Một mẫu training sẽ có cấu trúc như sau

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Sử dụng cánh tay robot để nhặt một nồi điện cụ thể và di chuyển nó đến một vị trí cụ thể. Nồi điện đó là nồi cơm điện màu đỏ, vị trí là tủ bếp gần đó.

### Input:


### Response:
Được thôi! Đầu tiên, cánh tay robot sẽ thu thập thông tin về tọa độ của vị trí hiện tại của nồi cơm điện màu đỏ và tủ bếp gần đó. Khi thông tin đã được nhập, cánh tay robot sẽ xoay về phía nồi cơm điện, kéo dài cánh tay để chạm đến nồi và đóng chốt xung quanh để giữ nó chắc chắn. Sau đó, cánh tay robot sẽ được kéo lại trong khi vẫn giữ chắc nồi cơm điện và cuối cùng xoay về phía tủ bếp gần đó. Cánh tay robot sẽ được hạ xuống để đặt nồi cơm điện vào tủ bếp và chốt được thả ra để bỏ nồi cơm điện đi. Sau đó, cánh tay robot sẽ thu lại vị trí ban đầu.

<|end_of_text|>

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=8,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,970 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 8
\        /    Total batch size = 128 | Total steps = 1,240
 "-____-"     Number of trainable parameters = 11,272,192
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Step,Training Loss
1,14.734200
2,14.895900
3,14.522800
4,14.291100
5,13.372200
6,12.617500
7,12.229200
8,11.802500
9,11.449700
10,11.188800
